# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [9]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [41]:
import dask.dataframe as dd

In [42]:
import os
from glob import glob
from dotenv import load_dotenv
load_dotenv()
# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
print("The number of parquet files are",len(parquet_files),"And the path is ",PRICE_DATA)

The number of parquet files are 558 And the path is  ../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [55]:
# Write your code below.
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
)).assign(hi_lo_range=lambda x: x['High']-x['Low'])
print(dd_feat.head())

C:\Users\clair\AppData\Local\Temp\ipykernel_29648\4280704272.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


                            Date       Open       High        Low      Close  \
ticker                                                                         
A      2013-12-02 00:00:00-05:00  34.875483  35.005930  34.640675  34.705898   
A      2013-12-03 00:00:00-05:00  34.516744  34.679807  34.281936  34.523266   
A      2013-12-04 00:00:00-05:00  34.464567  35.110291  34.386300  34.947227   
A      2013-12-05 00:00:00-05:00  34.797202  35.090711  34.647187  34.895039   
A      2013-12-06 00:00:00-05:00  35.064621  35.827743  35.064621  35.762520   

         Volume  Dividends  Stock Splits  year  Close_lag_1   returns  \
ticker                                                                  
A       2039962        0.0           0.0  2013          NaN       NaN   
A       3462706        0.0           0.0  2013    34.705898 -0.005262   
A       3377288        0.0           0.0  2013    34.523266  0.012280   
A       2530939        0.0           0.0  2013    34.947227 -0.001493   
A

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [54]:
# Write your code below.

dd_pd=dd_feat.compute()
dd_pd = (dd_pd.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(rolling_avg_return = x['returns'].rolling(10).mean())
))
dd_pd.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,positive_return,hi_lo_range,rolling_avg_return
ticker,,,,,,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.875483,35.005930,34.640675,34.705898,2039962,0.0,0.0,2013,NaN,NaN,0,0.365254,NaN
A,2013-12-03 00:00:00-05:00,34.516744,34.679807,34.281936,34.523266,3462706,0.0,0.0,2013,34.705898,-0.005262,0,0.397871,NaN
A,2013-12-04 00:00:00-05:00,34.464567,35.110291,34.386300,34.947227,3377288,0.0,0.0,2013,34.523266,0.012280,1,0.723990,NaN
A,2013-12-05 00:00:00-05:00,34.797202,35.090711,34.647187,34.895039,2530939,0.0,0.0,2013,34.947227,-0.001493,0,0.443525,NaN
A,2013-12-06 00:00:00-05:00,35.064621,35.827743,35.064621,35.762520,4268513,0.0,0.0,2013,34.895039,0.024860,1,0.763122,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It was not really necessary to convert pandas for calculating because dask have rolling calculation method.
Dask is great to use when we have a large data set, However it is slow.
I would use Pandas if I have small data set for the speed and memory. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.